In [2]:
import pybullet
import gym
import numpy as np
import matplotlib.pyplot as plt
import pybullet_data
import math
import rrtstar
import mpc
import time
import torch
from environment import Environment
from maddpg import MADDPGAgentTrainer
import tensorflow as tf
import tensorflow.keras.layers as layers

In [ ]:
#!git clone https://github.com/ROBOTIS-GIT/turtlebot3.git

In [3]:
# RRT
goal = (5.25, 3.5, 0)
start = (0.25, 0.25, 0)
samplingArea = [0, 10, 0, 5]
RRT = rrtstar.RRT(start, goal, [], samplingArea, 'rrtstar', 'rectangle', 2)
path = RRT.planning()

[0.25, 0.25, 0]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[3.8599826844913943, 4.847480331189188, 5.637115161681364]
[0. 0. 0.]
[3.8599826844913943, 4.847480331189188, 5.637115161681364]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[3.8599826844913943, 4.847480331189188, 5.637115161681364]
[0. 0. 0.]
[4.638396401952711, 2.6542846563973734, 2.689156489121335]
[0. 0. 0.]
[4.638396401952711, 2.6542846563973734, 2.689156489121335]
[0. 0. 0.]
[4.638396401952711, 2.6542846563973734, 2.689156489121335]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[3.8599826844913943, 4.847480331189188, 5.637115161681364]
[0. 0. 0.]
[4.638396401952711, 2.6542846563973734, 2.689156489121335]
[0. 0. 0.]
[5.25, 3.5, 0]
[0. 0. 0.]
[5.25, 3.5, 0]
[0. 0. 0.]
[5.25, 3.5, 0]
[0. 0. 0.]
[5.25, 3.5, 0]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[3.8599826844913943, 4.847480331189188, 5.637115161681364]
[0. 0. 0.]
[4.638396401952711, 2.6542846563973734, 2.689156489121335]
[0. 0. 0.]
[5.25, 3.5, 0]
[0. 0. 0.]
[8.418963411192768, 3.0686263166575

In [4]:
print(path)
path = np.flip(path, 0)
print(path)


[[5.25, 3.5, 0], [5.25, 3.5, 0], [0.25, 0.25, 0]]
[[0.25 0.25 0.  ]
 [5.25 3.5  0.  ]
 [5.25 3.5  0.  ]]


In [5]:
# interpolate path given by rrt
extended_path = []
for i in range(len(path)):
    print(path[i])
for i in range(len(path)):
    if i + 1 != len(path) - 1:
        angle = 0.39
        path_part = list(zip(np.linspace(path[i][0], path[i+1][0], 100), np.linspace(path[i][1], path[i+1][1], 100)))
        for part in path_part:
            extended_path.append((part[0], part[1], angle))
    else:
        break
        

[0.25 0.25 0.  ]
[5.25 3.5  0.  ]
[5.25 3.5  0.  ]


In [13]:
physicsClient = pybullet.connect(pybullet.GUI)
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())
pybullet.setGravity(0,0,-9.8)

# Loading chessgrid plane
planeID = pybullet.loadURDF('plane.urdf')
chess_grid = pybullet.loadTexture('checker_huge.gif')
pybullet.changeVisualShape(planeID, -1, textureUniqueId=chess_grid)

# Setting up overhead camera
viewMatrix = pybullet.computeViewMatrix(
    cameraEyePosition=[0.75,0.25,3],
    cameraTargetPosition=[0.75,0.25,0],
    cameraUpVector=[0,1,0])

projectionMatrix = pybullet.computeProjectionMatrixFOV(
    fov=45.0,
    aspect=1.0,
    nearVal=0.1,
    farVal=3.1)

width, height, rgbImg, depthImg, segImg = pybullet.getCameraImage(width = 224, height = 224, viewMatrix=viewMatrix, projectionMatrix = projectionMatrix)
player = pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', start, pybullet.getQuaternionFromEuler([0,0,0]))
pybullet.changeVisualShape(player, 0, rgbaColor=[0,0,1,1])

pybullet.setRealTimeSimulation(1)

In [7]:
# apply set of actions and return new states as a result of applying each action
def applyAction(actions, orig_state):
    new_states = []
    for i in range(G):
        action_set = actions[i]
        for j in range(H):
            action = action_set[j]
            action_3d = [fM * action[0], fM * action[1], 0]
            for x in range(10):
                pybullet.stepSimulation()
                time.sleep(1./240.)
                current_pos = pybullet.getBasePositionAndOrientation(player)[0]
                #pybullet.applyExternalForce(player, -1, action_3d, current_pos, pybullet.WORLD_FRAME)
                pybullet.resetBaseVelocity(player, action_3d)
            pybullet.resetBaseVelocity(player, [0,0,0])

        
        new_states.append(pybullet.getBasePositionAndOrientation(player))
        pybullet.resetBasePositionAndOrientation(player, orig_state[0], orig_state[1])
    return new_states
        

In [11]:
# get cost of actions
def getCost(states, target, actions):
    #target = [goal[0], goal[1], 0]
    target_pos = [target[0], target[1]]
    target_angle = target[2]
    costs = []
    for i in range(G):
        current = states[i]
        current_pos = current[0]
        current_angle = pybullet.getEulerFromQuaternion(current[1])[2]
        # Double check above. Based on angle used for initial setting position
        cost = math.sqrt(((current_pos[0] - target_pos[0]) ** 2) + ((current_pos[1] - target_pos[1]) ** 2))
        cost += abs(current_angle - target_angle)
        for action in actions[i]:
            if action[0] > 10 or action[1] > 10:
                cost += 10
        costs.append(cost)
    return costs

In [9]:
# planning horizon 
H = 5
# number of iterations
T = 5
# number of samples picked
K = 1
fM = 1
G = 10

def cem(path):
    pybullet.setRealTimeSimulation(0)
    total_actions = []
    # run loop for each coordinate in path
    for i in range(len(path) - 1):
    #for i in range(1000):
        initial_actions = np.random.normal(0,1,(G,H,2))
        state = pybullet.getBasePositionAndOrientation(player)
        print(state)
        for t in range(T):
            new_states = applyAction(initial_actions, state)
            costs = getCost(new_states, path[i + 1], initial_actions)
            #costs = getCost(new_states, path[1])
            # sort actions
            sorted_actions = [act for _,act in sorted(zip(costs, initial_actions), key=lambda pair: pair[0])]
            new_mean = np.mean(sorted_actions[:K])
            new_std = np.std(sorted_actions[:K])
            # get new actions from normal distribution with mean and std dev from previous actions
            new_actions = np.random.normal(new_mean, new_std, (G,H,2))
            #print(sorted_actions[:K])
            #print(new_actions)
            initial_actions = np.concatenate((sorted_actions[:K], new_actions))
        # pick the best action 
        best_action = initial_actions[0][0] # Because no horizon length
        action = [fM * best_action[0], fM * best_action[1], 0]
        total_actions.append(action)
        #print(action)
        # apply action to robot 
        for x in range(10):
            pybullet.stepSimulation()
            time.sleep(1./240.)
            #pybullet.applyExternalForce(player, -1, action, state[0], pybullet.WORLD_FRAME)
            pybullet.resetBaseVelocity(player, action)
        pybullet.resetBaseVelocity(player, [0,0,0])
    return total_actions

            
            


In [14]:
actions = cem(extended_path)
#pybullet.setRealTimeSimulation(1)

((0.24557051887589634, 0.24999802627452763, 0.0014247287237882265), (-4.829571848874629e-05, 0.08221814912468287, 0.001017893646274724, 0.9966138357029567))
((0.25561920864655785, 0.255390539489168, 0.0014343608430344207), (0.00015486846936998565, 0.08225922442271594, -0.0017011758425443083, 0.9966095032729134))
((0.2751429028574615, 0.27498627999461855, 0.001433981707156807), (0.0001095409131187327, 0.0822371433348424, -0.001312230086078617, 0.9966119196101931))
((0.3128462358779906, 0.23833518169888795, 0.0014131478731107359), (0.0005704689036110246, 0.08208205580701838, -0.006918400684819583, 0.9966013979579237))
((0.3311607317224835, 0.2730128048631386, 0.001391641880523574), (0.0004400252974245858, 0.0819254381566444, -0.0053417755717902965, 0.9966240486734805))
((0.3747470936754526, 0.2977182617432706, 0.0013839054919369297), (0.0005238974195849386, 0.08186983771761472, -0.006371500833405539, 0.9966225259247932))
((0.41115421508911765, 0.3390220661810983, 0.0013767147109127154), 

In [15]:
print(actions)

[[0.2792415222227355, 0.15898213731244953, 0], [0.5266470325403009, 0.5420394687261294, 0], [1.0118075574938428, -0.999271432126974, 0], [0.4918942636207741, 0.9454984536227434, 0], [1.1728910700763913, 0.6754332333068666, 0], [0.9764746969270649, 1.12169345875218, 0], [1.1635960657619582, 1.196679631214145, 0], [1.1476666358009422, 0.3723680048244462, 0], [0.7106038849220843, 0.8950015760036475, 0], [0.2589075691239704, 1.1516481633565636, 0], [2.1289606484236083, 1.0393005707210108, 0], [1.7434365365430335, -0.028246073098727065, 0], [0.9457096310617968, -0.7230233658563889, 0], [1.0233536151140408, 2.0101578919464624, 0], [1.6530630306898675, 0.5419496992066952, 0], [1.4380795207626869, 1.1468012496910305, 0], [0.753374336601423, 1.0827642155539081, 0], [1.315395250094356, 0.8711008781944944, 0], [2.3609237014402122, 1.5693836833292003, 0], [0.44667604006775136, 0.7110535329657983, 0], [0.7540386280512167, 0.4210155169721197, 0], [0.11606466206588784, 0.999921672668324, 0], [1.21867

In [17]:
# end simulation
pybullet.resetBasePositionAndOrientation(player, [0.25,0.25,0], pybullet.getQuaternionFromEuler([0,0,0]))
for action in actions:
    for x in range(10):
        pybullet.stepSimulation()
        time.sleep(1./240.)
        #pybullet.applyExternalForce(player, -1, action, state[0], pybullet.WORLD_FRAME)
        pybullet.resetBaseVelocity(player, action)
    pybullet.resetBaseVelocity(player, [0,0,0])

In [12]:
pybullet.disconnect()